## Yahoo options: Fetch options using unpublished yahoo rest api calls
### Usage:  _change the value of sym below, and run all cells_


In [1]:
import urllib.request
import json
import datetime
import pandas as pd
import numpy as np
import pathlib
from IPython import display
from tqdm import tqdm,tqdm_notebook




In [2]:
def _to_dt(ts):
    d = datetime.datetime.fromtimestamp(ts)
    return datetime.datetime(d.year,d.month,d.day)

_DAY_INDEX = ['MON','TUE','WED','THU','FRI','SAT','SUN']
def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    f = f'W-{_DAY_INDEX[target_weekday]}'
    dr = pd.date_range(datetime.datetime(int(year),int(month),1), periods=nth_occurrence, freq=f)[-1]
    return dr



In [21]:
def get_yahoo_options_expirations(sym):
    base_url = f'https://query1.finance.yahoo.com/v7/finance/options/{sym}'
    # get main list of contracts
    r =  urllib.request.urlopen(base_url)
    h = r.read().decode("utf-8")
    d = json.loads(h)
    c = d['optionChain']['result'][0]
    dict_ts =  {d:_to_dt(d) for d in c['expirationDates']}
    dict_dt =  {_to_dt(d):d for d in c['expirationDates']}
    dict_monthlies = {dt:dict_dt[dt] for dt in dict_dt.keys() if get_nth_weekday(dt.year,dt.month,3,3)==dt}
    return dict_ts,dict_dt,dict_monthlies


def get_yahoo_options(sym,expirations=None):
    base_url = f'https://query1.finance.yahoo.com/v7/finance/options/{sym}'
    # get main list of contracts
    r =  urllib.request.urlopen(base_url)
    h = r.read().decode("utf-8")
    d = json.loads(h)
    c = d['optionChain']['result'][0]
    print(c.keys())

    # now process chains
    df_chain = None
    expiries_to_get = c['expirationDates'] if expirations is None else expirations
    for e in expiries_to_get:
        u = base_url + f'?date={e}'
        print(u,end=' : ')
        r =  urllib.request.urlopen(u)
        h = r.read().decode("utf-8")
        # print(h)
        d = json.loads(h)
        calls = d['optionChain']['result'][0]['options'][0]['calls']
        puts = d['optionChain']['result'][0]['options'][0]['puts']
        chain = calls + puts
        print(f'calls: {len(calls)} puts:{len(puts)} all:{len(chain)}')
        df_temp = pd.DataFrame(chain)
        if df_chain is None:
            df_chain = df_temp.copy()
        else:
            df_chain = df_chain.append(df_temp)
    def get_dte(v):
        try:
            t= datetime.datetime.now()
            y = int(v.split(sym)[1][0:2]) + 2000
            m = int(v.split(sym)[1][2:4])
            d = int(v.split(sym)[1][4:6])
            dt = datetime.datetime(y,m,d)
            return (dt - t).days
        except:
            return None
    df_chain['dte'] = df_chain.contractSymbol.apply(get_dte)
    prev_close = c['quote']['regularMarketPreviousClose']
    df_chain['pc'] = df_chain.contractSymbol.str[-9]
    return df_chain

def get_first_n_expiries(sym,first_n_expiries=10):
    dict_exp,dict_dt,dict_monthlies = get_yahoo_options_expirations(sym)
    expiries = list(dict_dt.values())[:first_n_expiries]
    df = get_yahoo_options(sym)    
    return df

def get_yahoo_monthly_options(sym):
    dict_exp,dict_dt,dict_monthlies = get_yahoo_options_expirations(sym)
    monthlies = list(dict_monthlies.values())[:2]
    df_monthlies = get_yahoo_options(sym,expirations=monthlies)    
    return df_monthlies

def get_ctm_implied_vols(df_chain,pc='c'):
    pc2 = pc.upper()
    dfc2 = df_chain[(df_chain.inTheMoney==False) & (df_chain.pc==pc2)][['strike','expiration']]
    dfc3 = dfc2.groupby('expiration',as_index=False).min()
    dfc4 = dfc3.merge(df_chain[df_chain.pc==pc2],on=['expiration','strike'],how='inner')
    return dfc4

def get_avg_implied_vol_from_chain(df_chain,pc='c'):
    dfc2 = get_ctm_implied_vols(df_chain,pc=pc)
    dict_exp = {exp:_to_dt(exp) for exp in dfc2.expiration.values}
    def __is3t(dt):
        y = dt.year
        m = dt.month
        d = dt.day
        dt2 = datetime.datetime(y,m,d)
        thur3 = get_nth_weekday(y,m,3,3)
        return dt2==thur3

    third_thursdays = [k for k in dict_exp.keys() if __is3t(dict_exp[k])]
    dfc3 = dfc2[dfc2.expiration.isin(third_thursdays)].copy()
    dfc3['exp_dt']= [dict_exp[k] for k in dfc3.expiration.values]
    dfc4 = dfc3[dfc3.dte>=25].sort_values('dte')[:2]
    return dfc4

    

In [22]:
def get_implied_vols(sym_list,get_monthly_only=False,first_n_expiries=10):
    dict_ret = {}
    for sym in tqdm_notebook(sym_list):
        try:
            if get_monthly_only:
                df = get_yahoo_monthly_options(sym)
            else:
                df = get_first_n_expiries(sym,first_n_expiries=first_n_expiries)
            dict_ret[sym] = df.copy()
        except Exception as e:
            dict_ret[sym] = str(e)
            print(str(e))
    return dict_ret


In [4]:
def get_avg_implied_vols(sym_list):
    dict_ret = {}
    for sym in tqdm_notebook(sym_list):
        try:
            df = get_yahoo_monthly_options(sym)
            df = get_avg_implied_vol_from_chain(df)
            dict_ret[sym] = df.copy()
        except Exception as e:
            dict_ret[sym] = str(e)
            print(str(e))
    return dict_ret
        


In [7]:
h = pathlib.Path.home()
df_sp500 = pd.read_csv(f"{h}/downloads/sp500_constituents.csv")
dict_avg_implied_vols = get_avg_implied_vols(df_sp500.Symbol.values)

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1597968000 : calls: 26 puts:29 all:55
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1600387200 : calls: 30 puts:31 all:61
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/AOS?date=1597968000 : calls: 4 puts:5 all:9
https://query1.finance.yahoo.com/v7/finance/options/AOS?date=1600387200 : calls: 4 puts:2 all:6
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/ABT?date=1597968000 : calls: 54 puts:59 all:113
https://query1.finance.yahoo.com/v7/finance/options/ABT?date=1600387200 : calls: 17 puts:14 all:31
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://que

https://query1.finance.yahoo.com/v7/finance/options/AMCR?date=1597968000 : calls: 6 puts:5 all:11
https://query1.finance.yahoo.com/v7/finance/options/AMCR?date=1600387200 : calls: 4 puts:2 all:6
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/AEE?date=1597968000 : calls: 6 puts:7 all:13
https://query1.finance.yahoo.com/v7/finance/options/AEE?date=1600387200 : calls: 8 puts:10 all:18
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/AAL?date=1597968000 : calls: 54 puts:54 all:108
https://query1.finance.yahoo.com/v7/finance/options/AAL?date=1600387200 : calls: 19 puts:19 all:38
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/AEP?date=1597968000 : calls: 24 puts:26 all:50
https://quer

https://query1.finance.yahoo.com/v7/finance/options/ATO?date=1600387200 : calls: 3 puts:1 all:4
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/ADSK?date=1597968000 : calls: 29 puts:31 all:60
https://query1.finance.yahoo.com/v7/finance/options/ADSK?date=1600387200 : calls: 10 puts:15 all:25
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/ADP?date=1597968000 : calls: 27 puts:34 all:61
https://query1.finance.yahoo.com/v7/finance/options/ADP?date=1600387200 : calls: 9 puts:9 all:18
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/AZO?date=1597968000 : calls: 40 puts:37 all:77
https://query1.finance.yahoo.com/v7/finance/options/AZO?date=1600387200 : calls: 70 puts:59 all:129
dict_keys

https://query1.finance.yahoo.com/v7/finance/options/COF?date=1597968000 : calls: 43 puts:49 all:92
https://query1.finance.yahoo.com/v7/finance/options/COF?date=1600387200 : calls: 41 puts:39 all:80
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/CAH?date=1597968000 : calls: 21 puts:22 all:43
https://query1.finance.yahoo.com/v7/finance/options/CAH?date=1600387200 : calls: 18 puts:17 all:35
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/KMX?date=1597968000 : calls: 37 puts:34 all:71
https://query1.finance.yahoo.com/v7/finance/options/KMX?date=1600387200 : calls: 12 puts:10 all:22
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/CCL?date=1597968000 : calls: 35 puts:32 all:67
https:/

...

dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/RL?date=1597968000 : calls: 11 puts:12 all:23
https://query1.finance.yahoo.com/v7/finance/options/RL?date=1600387200 : calls: 6 puts:6 all:12
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/RJF?date=1597968000 : calls: 23 puts:28 all:51
https://query1.finance.yahoo.com/v7/finance/options/RJF?date=1600387200 : calls: 5 puts:2 all:7
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/RTX?date=1597968000 : calls: 39 puts:34 all:73
https://query1.finance.yahoo.com/v7/finance/options/RTX?date=1600387200 : calls: 15 puts:13 all:28
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query

https://query1.finance.yahoo.com/v7/finance/options/SNA?date=1600387200 : calls: 21 puts:24 all:45
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/SO?date=1597968000 : calls: 30 puts:39 all:69
https://query1.finance.yahoo.com/v7/finance/options/SO?date=1600387200 : calls: 7 puts:7 all:14
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/LUV?date=1597968000 : calls: 35 puts:34 all:69
https://query1.finance.yahoo.com/v7/finance/options/LUV?date=1600387200 : calls: 25 puts:25 all:50
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/SWK?date=1597968000 : calls: 18 puts:20 all:38
https://query1.finance.yahoo.com/v7/finance/options/SWK?date=1600387200 : calls: 9 puts:8 all:17
dict_keys(['u

https://query1.finance.yahoo.com/v7/finance/options/TRV?date=1597968000 : calls: 12 puts:14 all:26
https://query1.finance.yahoo.com/v7/finance/options/TRV?date=1600387200 : calls: 6 puts:8 all:14
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1597968000 : calls: 44 puts:42 all:86
https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1600387200 : calls: 34 puts:34 all:68
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/TMO?date=1597968000 : calls: 18 puts:21 all:39
https://query1.finance.yahoo.com/v7/finance/options/TMO?date=1600387200 : calls: 29 puts:34 all:63
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/TIF?date=1597968000 : calls: 20 puts:18 all:38
https://q

dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/VRSN?date=1597968000 : calls: 14 puts:14 all:28
https://query1.finance.yahoo.com/v7/finance/options/VRSN?date=1600387200 : calls: 26 puts:26 all:52
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/VRSK?date=1597968000 : calls: 12 puts:10 all:22
https://query1.finance.yahoo.com/v7/finance/options/VRSK?date=1600387200 : calls: 20 puts:18 all:38
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/VZ?date=1597968000 : calls: 21 puts:25 all:46
https://query1.finance.yahoo.com/v7/finance/options/VZ?date=1600387200 : calls: 23 puts:22 all:45
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
http

dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/XLNX?date=1597968000 : calls: 29 puts:36 all:65
https://query1.finance.yahoo.com/v7/finance/options/XLNX?date=1600387200 : calls: 21 puts:18 all:39
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/XYL?date=1597968000 : calls: 7 puts:7 all:14
https://query1.finance.yahoo.com/v7/finance/options/XYL?date=1600387200 : calls: 5 puts:3 all:8
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/YUM?date=1597968000 : calls: 18 puts:31 all:49
https://query1.finance.yahoo.com/v7/finance/options/YUM?date=1600387200 : calls: 7 puts:6 all:13
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://que

In [8]:
np.isnan(dict_avg_implied_vols['NWS'].impliedVolatility.mean())

True

In [9]:
df_avg_implied = None
for k in dict_avg_implied_vols.keys():
    try:
        df_avg_imp = dict_avg_implied_vols[k]
        avg_vol = df_avg_imp.impliedVolatility.mean()
        if np.isnan(avg_vol):
            continue
        df_avg_imp = pd.DataFrame({'sym':[k],'avg_vol':[avg_vol]})
        if df_avg_implied is None:
            df_avg_implied = df_avg_imp.copy()
        else:
            df_avg_implied = df_avg_implied.append(df_avg_imp)
        df_avg_implied.index = list(range(len(df_avg_implied)))
    except:
        print(f"{k}: {dict_avg_implied_vols[k]}")
        
# df_avg_implied = df_avg_implied[['sym','impliedVolatility']].groupby('sym',as_index=False).mean()

df_avg_implied = df_avg_implied.sort_values('avg_vol',ascending=False)
df_avg_implied.index = list(range(len(df_avg_implied)))
df_avg_implied
                                

BRK.B: 'NoneType' object has no attribute 'contractSymbol'
BF.B: 'NoneType' object has no attribute 'contractSymbol'
NVR: 'NoneType' object has no attribute 'contractSymbol'


,sym,avg_vol
0,NCLH,0.964847
1,COTY,0.916016
2,APA,0.897950
3,AAL,0.894044
4,RCL,0.876832
...,...,...
496,BAX,0.127018
497,LNT,0.125736
498,HSY,0.112064
499,FE,0.000000


In [10]:
df_avg_implied.to_csv('df_avg_implied.csv',index=False)

In [23]:
h = pathlib.Path.home()
df_sp500 = pd.read_csv(f"{h}/downloads/sp500_constituents.csv")
dict_all_implied_vols = get_implied_vols(df_sp500.Symbol.values)

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1596153600 : calls: 33 puts:24 all:57
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1596758400 : calls: 25 puts:23 all:48
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1597363200 : calls: 17 puts:25 all:42
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1597968000 : calls: 26 puts:29 all:55
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1598572800 : calls: 20 puts:22 all:42
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1599177600 : calls: 2 puts:4 all:6
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1600387200 : calls: 30 puts:31 all:61
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1602806400 : calls: 25 puts:30 all:55
https://query1.finance.yahoo.com/v7/finance/options/MMM?date=1605830400 : calls: 27 puts:24 all:51
https://quer

https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1597363200 : calls: 47 puts:41 all:88
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1597968000 : calls: 77 puts:67 all:144
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1598572800 : calls: 49 puts:44 all:93
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1599177600 : calls: 4 puts:11 all:15
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1600387200 : calls: 42 puts:35 all:77
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1602806400 : calls: 78 puts:70 all:148
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1610668800 : calls: 89 puts:83 all:172
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1623974400 : calls: 49 puts:46 all:95
https://query1.finance.yahoo.com/v7/finance/options/ADBE?date=1642723200 : calls: 60 puts:59 all:119
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'

https://query1.finance.yahoo.com/v7/finance/options/ALB?date=1610668800 : calls: 25 puts:22 all:47
https://query1.finance.yahoo.com/v7/finance/options/ALB?date=1616112000 : calls: 9 puts:7 all:16
https://query1.finance.yahoo.com/v7/finance/options/ALB?date=1642723200 : calls: 21 puts:18 all:39
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/ARE?date=1597968000 : calls: 7 puts:11 all:18
https://query1.finance.yahoo.com/v7/finance/options/ARE?date=1600387200 : calls: 5 puts:2 all:7
https://query1.finance.yahoo.com/v7/finance/options/ARE?date=1602806400 : calls: 15 puts:22 all:37
https://query1.finance.yahoo.com/v7/finance/options/ARE?date=1610668800 : calls: 14 puts:13 all:27
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/ALXN?date=1596153600 : calls: 29 puts:22 all:51
https://query

dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1596153600 : calls: 28 puts:28 all:56
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1596758400 : calls: 23 puts:20 all:43
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1597363200 : calls: 18 puts:22 all:40
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1597968000 : calls: 26 puts:27 all:53
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1598572800 : calls: 22 puts:20 all:42
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1599177600 : calls: 4 puts:0 all:4
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1600387200 : calls: 19 puts:20 all:39
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1608249600 : calls: 16 puts:18 all:34
https://query1.finance.yahoo.com/v7/finance/options/MO?date=1610668800 : calls: 28 puts:28 all:56
https://query1.financ

...

https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1610668800 : calls: 34 puts:31 all:65
https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1623974400 : calls: 28 puts:31 all:59
https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1642723200 : calls: 31 puts:29 all:60
https://query1.finance.yahoo.com/v7/finance/options/DIS?date=1655424000 : calls: 26 puts:26 all:52
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
'NoneType' object has no attribute 'contractSymbol'
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/TIF?date=1596153600 : calls: 9 puts:7 all:16
https://query1.finance.yahoo.com/v7/finance/options/TIF?date=1596758400 : calls: 2 puts:4 all:6
https://query1.finance.yahoo.com/v7/finance/options/TIF?date=1597363200 : calls: 1 puts:3 all:4
https://query1.finance.yahoo.com/v7/finance/options/TIF?date=15

https://query1.finance.yahoo.com/v7/finance/options/USB?date=1610668800 : calls: 23 puts:24 all:47
https://query1.finance.yahoo.com/v7/finance/options/USB?date=1616112000 : 'NoneType' object is not subscriptable
HTTP Error 404: Not Found
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
'NoneType' object has no attribute 'contractSymbol'
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/UAA?date=1596153600 : calls: 14 puts:12 all:26
https://query1.finance.yahoo.com/v7/finance/options/UAA?date=1596758400 : calls: 13 puts:13 all:26
https://query1.finance.yahoo.com/v7/finance/options/UAA?date=1597363200 : calls: 10 puts:8 all:18
https://query1.finance.yahoo.com/v7/finance/options/UAA?date=1597968000 : calls: 15 puts:15 all:30
https://query1.finance.yahoo.com/v7/finance/options/UAA?date=1598572800 : calls: 3 puts:8 all:11
https://query1.fin

https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1597968000 : calls: 33 puts:29 all:62
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1598572800 : calls: 26 puts:27 all:53
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1599177600 : calls: 0 puts:0 all:0
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1600387200 : calls: 20 puts:21 all:41
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1608249600 : calls: 13 puts:15 all:28
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1610668800 : calls: 0 puts:0 all:0
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1616112000 : calls: 6 puts:6 all:12
https://query1.finance.yahoo.com/v7/finance/options/VLO?date=1642723200 : calls: 27 puts:0 all:27
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/VAR?date=1597968000 : calls: 15 puts:13 all:28
https://query1.fin

https://query1.finance.yahoo.com/v7/finance/options/WAT?date=1608249600 : calls: 14 puts:17 all:31
https://query1.finance.yahoo.com/v7/finance/options/WAT?date=1613692800 : calls: 9 puts:6 all:15
HTTP Error 404: Not Found
dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1596153600 : calls: 33 puts:31 all:64
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1596758400 : calls: 26 puts:27 all:53
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1597363200 : calls: 22 puts:22 all:44
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1597968000 : calls: 32 puts:29 all:61
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1598572800 : calls: 26 puts:28 all:54
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1599177600 : calls: 14 puts:9 all:23
https://query1.finance.yahoo.com/v7/finance/options/WFC?date=1600387200 : calls: 10 p

dict_keys(['underlyingSymbol', 'expirationDates', 'strikes', 'hasMiniOptions', 'quote', 'options'])
'NoneType' object has no attribute 'contractSymbol'



In [28]:
df_all_implied = None
for k in tqdm_notebook(dict_all_implied_vols.keys()):
    try:
        df_all_imp  = dict_all_implied_vols[k]
        avg_vol = df_all_imp.impliedVolatility.mean()
        if np.isnan(avg_vol):
            continue
        df_all_imp['symbol'] = k
        if df_all_implied is None:
            df_all_implied = df_all_imp.copy()
        else:
            df_all_implied = df_all_implied.append(df_all_imp)
        df_all_implied.index = list(range(len(df_all_implied)))
    except:
        print(f"{k}: {dict_all_implied_vols[k]}")
        


/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


BRK.B: 'NoneType' object has no attribute 'contractSymbol'
BF.B: 'NoneType' object has no attribute 'contractSymbol'
INTU: 'NoneType' object has no attribute 'contractSymbol'
ISRG: HTTP Error 404: Not Found
IPGP: 'NoneType' object has no attribute 'contractSymbol'
JKHY: HTTP Error 404: Not Found
JNJ: 'NoneType' object has no attribute 'contractSymbol'
JPM: 'NoneType' object is not subscriptable
JNPR: 'NoneType' object is not subscriptable
KEYS: HTTP Error 404: Not Found
KMB: 'NoneType' object is not subscriptable
KIM: 'NoneType' object has no attribute 'contractSymbol'
KR: HTTP Error 404: Not Found
LH: 'NoneType' object has no attribute 'contractSymbol'
LRCX: 'NoneType' object is not subscriptable
LVS: 'NoneType' object is not subscriptable
LKQ: HTTP Error 404: Not Found
LOW: 'NoneType' object is not subscriptable
LYB: 'NoneType' object is not subscriptable
MKTX: 'NoneType' object has no attribute 'contractSymbol'
MMC: HTTP Error 404: Not Found
MA: 'NoneType' object has no attribute 'c

In [ ]:
#!jupyter nbconvert --to script yahoo_options.ipynb

In [33]:
df_all_implied.to_csv('df_all_implied.csv',index=False)